# Leonardo's Mechanical Drum: The Rhythm of War

> *"Music is the shaping of the invisible..."*  
> — Leonardo da Vinci

## Introduction
Leonardo designed a mechanical drum to accompany soldiers in battle. It used a rotating drum with pegs to lift and drop drumsticks, creating complex rhythmic patterns automatically.

In this notebook, we explore:
1.  **Mechanical Sequencing**: How pegs on a cylinder create a timeline of beats.
2.  **Rhythm Generation**: Visualizing the patterns created by different peg arrangements.
3.  **Timing Accuracy**: The effect of mechanical tolerances on the "groove".

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. The Timeline
The drum rotates at a constant angular velocity $\omega$. The position of a peg $\theta$ determines when it strikes:
$$ t_{strike} = \frac{\theta}{\omega} $$

### 2. Polyrhythms
By having multiple rows of pegs striking different drums (or different parts of the same drum), Leonardo could create complex polyrhythms (e.g., 3 against 4).

In [ ]:
def generate_rhythm(bpm, pattern_type, jitter_ms):
    # 1 beat = 60/bpm seconds
    beat_duration = 60.0 / bpm
    
    # Define patterns (1 = hit, 0 = rest)
    # 16 steps (1 measure of 4/4 sixteenth notes)
    if pattern_type == 'March':
        # ONE two three four
        pattern = [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
    elif pattern_type == 'Gallop':
        # da-da-DA da-da-DA
        pattern = [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]
    elif pattern_type == 'Syncopated':
        # 3-3-3-3-2-2 (Tresillo-ish)
        pattern = [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
        
    # Generate timestamps
    times = []
    hits = []
    
    step_duration = beat_duration / 4.0 # 16th notes
    
    for i, hit in enumerate(pattern):
        if hit:
            # Ideal time
            t = i * step_duration
            # Add mechanical jitter (noise)
            noise = np.random.normal(0, jitter_ms / 1000.0)
            t += noise
            times.append(t)
            hits.append(1)
            
    return times, hits, beat_duration * 4.0 # Measure duration

def plot_drum(bpm, pattern, jitter):
    times, hits, duration = generate_rhythm(bpm, pattern, jitter)
    
    fig, ax = plt.subplots(figsize=(12, 4))
    
    # Plot hits as vertical lines (sticks)
    ax.vlines(times, 0, 1, colors='k', linewidth=3, label='Drum Strike')
    
    # Visualize the "Grid" (Ideal 16th notes)
    grid_times = np.arange(0, duration, duration/16)
    for t in grid_times:
        ax.axvline(t, color='gray', linestyle=':', alpha=0.5)
        
    ax.set_xlim(-0.1, duration + 0.1)
    ax.set_ylim(0, 1.2)
    ax.set_yticks([])
    ax.set_xlabel('Time (s)')
    ax.set_title(f'Mechanical Drum Pattern: {pattern} ({bpm} BPM)')
    
    # Annotate Jitter effect
    if jitter > 20:
        ax.text(duration/2, 1.1, "High Mechanical Jitter! (Loose Gears)", 
                ha='center', color='red', fontweight='bold')
    
    plt.show()
    
    print(f"Total Hits per Measure: {len(hits)}")
    print(f"Measure Duration: {duration:.2f} s")

interact(plot_drum, 
         bpm=widgets.IntSlider(min=60, max=180, step=10, value=120, description='Tempo (BPM)'),
         pattern=widgets.Dropdown(options=['March', 'Gallop', 'Syncopated'], value='March', description='Pattern'),
         jitter=widgets.IntSlider(min=0, max=50, step=5, value=5, description='Jitter (ms)'));

## Conclusion

The mechanical drum is a **sequencer**. Just like a modern drum machine or DAW (Digital Audio Workstation), it places events on a timeline. The precision of the machine determines the "tightness" of the rhythm. Leonardo's designs often included gears to multiply the speed, allowing for rapid rolls that a human drummer might struggle to sustain.